In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import random
import os
#torch.manual_seed(1)

In [2]:
device = torch.device("cuda")

#remove_labels = ['roomno', 'cellno', 'community', 'devZone', 'subroadno', 'floorno', 'country', 'otherinfo']


T0 = ['redundant']
T1 = ['town', 'poi', 'assist']
T2 = ['houseno', 'city', 'district', 'road', 'roadno', 'subpoi', 'subRoad', 'person']
T3 = ['prov']
T4 = ['roomno', 'cellno', 'community', 'devZone', 'subroadno', 'floorno', 'country', 'otherinfo']
remove_labels = T4

max_train_epochs = 6
warmup_proportion = 0.05
gradient_accumulation_steps = 1
train_batch_size = 128
valid_batch_size = train_batch_size
test_batch_size = train_batch_size
data_workers= 6
# save_checkpoint = False

learning_rate= 1e-3
# weight_decay=0.01
# max_grad_norm=1.0

START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 200
HIDDEN_DIM = 32
# predict_max_label_len = 10
    
cur_time = time.strftime("%Y-%m-%d_%H:%M:%S")

In [3]:
olabels = ['B-assist', 'I-assist', 'B-cellno', 'I-cellno', 'B-city', 'I-city', 'B-community', 'I-community', 'B-country', 'I-country', 'B-devZone', 'I-devZone', 'B-district', 'I-district', 'B-floorno', 'I-floorno', 'B-houseno', 'I-houseno', 'B-otherinfo', 'I-otherinfo', 'B-person', 'I-person', 'B-poi', 'I-poi', 'B-prov', 'I-prov', 'B-redundant', 'I-redundant', 'B-road', 'I-road', 'B-roadno', 'I-roadno', 'B-roomno', 'I-roomno', 'B-subRoad', 'I-subRoad', 'B-subRoadno', 'I-subRoadno', 'B-subpoi', 'I-subpoi', 'B-subroad', 'I-subroad', 'B-subroadno', 'I-subroadno', 'B-town', 'I-town']
# print(len(labels))
# for l in remove_labels:
#     idx = labels.index('B-'+l)
#     labels.pop(idx)

# label2id = {}
olabels2id = {}
for i, l in enumerate(olabels):
    olabels2id[l] = i
labels = ['B-prov', 'I-prov', 'B-city', 'I-city', 'B-district', 'I-district', 'B-town', 'I-town',  'I-community', 'B-road', 'I-road', 'B-roadno', 'I-roadno', 'B-poi', 'I-poi', 'B-houseno', 'I-houseno', 'I-cellno', 'I-floorno', 'I-roomno', 'B-assist', 'I-assist', 'I-country', 'I-devZone', 'I-otherinfo', 'B-person', 'I-person', 'B-redundant', 'I-redundant', 'B-subpoi', 'I-subpoi', 'B-subroad', 'I-subroad', 'I-subroadno', ]
print(len(labels))
num_labels = len(labels)

label2id = {}
for i, l in enumerate(labels):
    label2id[l] = i
print(label2id)

34
{'B-prov': 0, 'I-prov': 1, 'B-city': 2, 'I-city': 3, 'B-district': 4, 'I-district': 5, 'B-town': 6, 'I-town': 7, 'I-community': 8, 'B-road': 9, 'I-road': 10, 'B-roadno': 11, 'I-roadno': 12, 'B-poi': 13, 'I-poi': 14, 'B-houseno': 15, 'I-houseno': 16, 'I-cellno': 17, 'I-floorno': 18, 'I-roomno': 19, 'B-assist': 20, 'I-assist': 21, 'I-country': 22, 'I-devZone': 23, 'I-otherinfo': 24, 'B-person': 25, 'I-person': 26, 'B-redundant': 27, 'I-redundant': 28, 'B-subpoi': 29, 'I-subpoi': 30, 'B-subroad': 31, 'I-subroad': 32, 'I-subroadno': 33}


In [4]:
word_embedding_file = 'neural-chinese-address-parsing/data/giga.vec100'
word2vec = {}
with open(word_embedding_file) as ff:
    for l in ff:
        l = l.strip().split(' ')
        word2vec[l[0]] = [float(x) for x in l[1:]]
print(len(word2vec))

6082


# 载入数据

In [5]:
def get_data_list(fn):
    with open(fn) as f:
        data_list = []  # 空的数据列表
        token, label = [], []  # 当前数据的字符和标签序列
        for l in f:
            l = l.strip().split()
            if not l:  # 如果 l 为空，说明当前数据结束了
                data_list.append([token, label])
                token, label = [], []
                continue
            token.append(l[0])
            label.append(l[1])
        assert len(token) == 0  # 数据最后一行应该是空行
    return data_list

In [6]:

train_list = get_data_list('neural-chinese-address-parsing/data/train.txt')
test_list = get_data_list('neural-chinese-address-parsing/data/dev.txt')
dev_list = get_data_list('neural-chinese-address-parsing/data/test.txt')
print(len(train_list), len(test_list), len(dev_list))
max_token_len = 0
for ls in [train_list, test_list, dev_list]:
    for l in ls:
        max_token_len = max(max_token_len, len(l[0]))
print('max_token_len', max_token_len)

8957 2985 2985
max_token_len 67


In [7]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        example = self.examples[index]
        sentence = example[0]

        label = example[1]

        sentence = [x.lower() for x in sentence]
        input_ids = []
        for ch in sentence:
            if ch in word2vec:
                input_ids.append(word2vec[ch])
            else:
                input_ids.append([random.random()/10 for _ in range(EMBEDDING_DIM)])
        return input_ids, label, index
ddd = []
bbb = []
def the_collate_fn(batch):
    #print(batch)
    input_len = [len(b[0]) for b in batch]
    max_len = max(input_len)
    x = [b[0] + [[0] * EMBEDDING_DIM for _ in range(max_len-len(b[0]))] for b in batch]
    input_ids = torch.Tensor(x)
    label = torch.LongTensor([b[1] + [0] * (max_len-len(b[0])) for b in batch])
    indexs = [b[2] for b in batch]
    return input_ids, label, input_len, indexs

train_dataset = MyDataSet(train_list)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle = True,
    num_workers=data_workers,
    collate_fn=the_collate_fn,
)

In [8]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [9]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True, batch_first=True)
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size).to(device))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden(train_batch_size)

    def init_hidden(self, batch_size):
        return (torch.randn(2, batch_size, self.hidden_dim // 2).to(device),
                torch.randn(2, batch_size, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats, input_len):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for i in range(input_len):
            feat = feats[i]
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden(len(sentence))
        embeds = sentence # sentence.view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print(lstm_out.shape)
        #lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = []
        for f in lstm_out:
            lstm_feats.append(self.hidden2tag(f))
        return lstm_feats

    def _score_sentence(self, feats, tags, input_len):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i in range(input_len):
            feat = feats[i]
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats, input_len):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for i in range(input_len):
            feat = feats[i]
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags, input_len):
        feats = self._get_lstm_features(sentence)
        batch_size = len(sentence)
        loss = None
        for b in range(batch_size):
            forward_score = self._forward_alg(feats[b], input_len[b])
            gold_score = self._score_sentence(feats[b], tags[b], input_len[b])
            batch_loss = forward_score - gold_score
            if loss is None:
                loss = batch_loss
            else:
                loss += batch_loss
        loss = loss/batch_size
        return loss

    def forward(self, sentence, input_len):  # dont confuse this with _forward_alg above.
        lstm_feats = self._get_lstm_features(sentence)
        res_logits = []
        batch_size = len(sentence)
        for b in range(batch_size):
            score, tag_seq = self._viterbi_decode(lstm_feats[b], input_len[b])
            res_logits.append(tag_seq)
        return res_logits

In [10]:
# Make up some training data

label2id[START_TAG] = len(labels)
labels.append(START_TAG)
label2id[STOP_TAG] = len(labels)
labels.append(STOP_TAG)

In [ ]:
model = BiLSTM_CRF(label2id, EMBEDDING_DIM, HIDDEN_DIM)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Check predictions before training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
#     print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(max_train_epochs):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags, input_len, ids in tqdm(train_data_loader):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        # Step 3. Run our forward pass.
        sentence = sentence.to(device)
        tags = tags.to(device)
        loss = model.neg_log_likelihood(sentence, tags, input_len)
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

  0%|          | 0/70 [00:00<?, ?it/s]